In [2]:
import pandas as pd
from typing import Dict
import warnings
warnings.filterwarnings("ignore")

In [8]:
df_playtime_genre = pd.read_csv('C:/Users/Daniel/Downloads/PI_01_07/Data/Df_funciones/playtime_genre.csv', encoding='utf-8')
df_user_for_genre = pd.read_csv('C:/Users/Daniel/Downloads/PI_01_07/Data/Df_funciones/user_for_genre.csv', encoding='utf-8')
df_user_recommend = pd.read_csv('C:/Users/Daniel/Downloads/PI_01_07/Data/Df_funciones/user_recommend.csv', encoding='utf-8')
df_sentiment_year = pd.read_csv('C:/Users/Daniel/Downloads/PI_01_07/Data/Df_funciones/sentiment_year.csv', encoding= 'utf-8')
df_reviews_clean = pd.read_csv('C:/Users/Daniel/Downloads/PI_01_07/Data/Df_funciones/reviews_clean.csv', encoding= 'utf-8')

In [9]:
df_reviews_clean

,posted,item_id,recommend,review,user_id,análisis_sentimiento
0,2011,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2
1,2011,22200,True,It's unique and worth a playthrough.,76561197970982479,2
2,2011,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2
3,2014,251610,True,I know what you think when you see this title ...,js41637,2
4,2013,227300,True,For a simple (it's actually not all that simpl...,js41637,0
...,...,...,...,...,...,...
49181,2015,730,True,its FUNNNNNNNN,wayfeng,1
49182,2015,253980,True,Awesome fantasy game if you don't mind the gra...,76561198251004808,2
49183,2015,730,True,Prettyy Mad Game,72947282842,0
49184,2015,730,True,AMAZING GAME 10/10,ApxLGhost,2


PLAYTIMEGENRE

In [21]:
df_playtime_genre.head(5)

,genres,date,playtime_forever
0,Action,1983,3473
1,Action,1984,384
2,Action,1988,16001
3,Action,1989,607
4,Action,1990,18271


In [15]:

# 1 FUNCION PLAYTIMEGENRE
def PlayTimeGenre(genre: str):

    ### Creo un dataset solo con el listado de datos con el genero brindado
    df_genero = df_playtime_genre[df_playtime_genre['genres'].str.lower() == genre.lower() if isinstance(genre, str) else None]

    ### Si el df está vacio, es porque no encontró informacion para el genero ingreado
    if df_genero.empty:
        return {f"No hay informacion para el genero {genre}": None}

    if not df_genero['playtime_forever'].empty:
        
        max_hours = df_genero.loc[df_genero['playtime_forever'].idxmax(), 'date']

        return {f"Año de lanzamiento con mayor horas de juego para el genero {genre}": max_hours}
    else:
        return {f"No hay informacion para el genero {genre}": None}

In [16]:
PlayTimeGenre('Action')

{'Año de lanzamiento con mayor horas de juego para el genero Action': 2012}

USER FOR GENRE

In [18]:
df_user_for_genre.head(5)

,genres,user_id,date,playtime_hours
0,Action,--000--,2009,88.81
1,Action,--000--,2010,0.37
2,Action,--000--,2012,1822.44
3,Action,--000--,2014,63.79
4,Action,--000--,2016,1.58


In [19]:
def UserForGenre(genre: str):
    
    genre_df = df_user_for_genre[df_user_for_genre['genres'].str.lower() == genre.lower() if isinstance(genre, str) else None]

    if genre_df.empty:
        return {"Usuario con mas horas jugadas para el genero": None, "Horas jugadas por año": {}}

    user_playtime_sum = genre_df.groupby('user_id')['horas'].sum()

    max_horas = user_playtime_sum.idxmax()

    usuario = genre_df[genre_df['user_id'] == max_horas]

    horas_año = dict(zip(usuario['date'], usuario['horas']))

    return {f"Usuario con mas horas jugadas para el genero {genre}": max_horas, "Horas jugadas por año": horas_año}

In [20]:
UserForGenre('2D')

{'Usuario con mas horas jugadas para el genero': None,
 'Horas jugadas por año': {}}

USER RECOMMEND

In [22]:
df_user_recommend.head(5)

,item_id,item_name,posted,recommend,análisis_sentimiento
0,70,Half-Life,2015,True,0
1,70,Half-Life,2011,True,0
2,70,Half-Life,2014,True,0
3,70,Half-Life,2013,True,2
4,70,Half-Life,2013,True,0


In [44]:
def UsersRecommend(year):

    year = str(year)
    
    reviews = df_user_recommend[(df_user_recommend['posted'] == year) & (df_user_recommend['recommend'] == True)]
    
    positive_reviews = reviews[reviews['análisis_sentimiento'].isin([1, 2])]
    
    count = positive_reviews['item_id'].value_counts().reset_index()
    count.columns = ['item_id', 'recommendations_count']

    top3 = count.head(3)
    
    result = [{"Posicion {}: {}".format(i+1, row['item_id']): row['recommendations_count']} for i, row in top3.iterrows()]
    
    return result 

In [45]:
UsersRecommend(2010)

[]

In [41]:
UsersRecommend(2010)

[]

USER NOT RECOMMEND

In [30]:
def UsersNotRecommend(year):
    
    year = str(year)
   

    filtered_reviews = df_user_recommend[(df_user_recommend['posted'] == year) & (df_user_recommend['recommend'] == False)]
    
    negative_reviews = filtered_reviews[filtered_reviews['análisis_sentimiento'] == 0]
    
    count = negative_reviews['item_id'].value_counts().reset_index()
    count.columns = ['item_id', 'count']
    
    top3_not_recommendations = count.head(3)
    
    
    result = [{"Posicion {}: {}".format(i+1, row['item_id']): row['count']} for i, row in top3_not_recommendations.iterrows()]
    
    return result


In [31]:
UsersNotRecommend(2014)

[]

In [39]:
def sentiment_analysis(year):

    year = str(year)

    filtered_reviews = df_sentiment_year[df_sentiment_year['date'] == year]
   
    sentiment_counts = filtered_reviews['análisis_sentimiento'].value_counts().to_dict()
  
    result = {
        'Negative': sentiment_counts.get(0, 0),
        'Neutral': sentiment_counts.get(1, 0),
        'Positive': sentiment_counts.get(2, 0)
    }
    
    return result

In [40]:
sentiment_analysis(2014)

{'Negative': 0, 'Neutral': 0, 'Positive': 0}